In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

In [ ]:
import minsearch
import os
from openai import OpenAI
import json

In [ ]:
# os.environ['HF_HOME'] = '/run/cache/'

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
docs_raw

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(text_fields=["question", "text", "section"],
                keyword_fields=["course"]
               )
index.fit(documents)

In [ ]:
'SELECT * WHERE course = "data-engineering-zoomcamp";'

In [ ]:
client = OpenAI()

In [ ]:
def search(query):
    boost = {"question": 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the
    CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': prompt}]
    )

    return response.choices[0].message.content

In [ ]:
query = "The course has already started can i enroll?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    response = llm(prompt)
    return response

In [ ]:
rag(query)

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = "I just discovered the course can i still join it?"

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    answer = es_client.search(index=index_name, body=search_query)

    answer_docs = []

    for hit in answer['hits']['hits']:
        answer_docs.append(hit['_source'])
    return answer_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    response = llm(prompt)
    return response

In [ ]:
rag(query)